In [1]:
import numpy as np
import pandas as pd
from scipy.stats import chi2_contingency
import matplotlib.pyplot as plt
from scipy.stats import mannwhitneyu
import pandas as pd
from statsmodels.stats.proportion import proportions_ztest

import matplotlib.pyplot as plt
import seaborn as sns


/Users/mahdie/opt/anaconda3/envs/physioai/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


#### Physical activity

In [2]:
dfPA_treatment = pd.read_csv("testdata/2.PA_treatment.csv")
dfPA_balanced = pd.read_csv("testdata/3.PA_balanced.csv").drop(columns='Treatment')
dfPA_control = pd.read_csv("testdata/2.PA_control.csv")

print("Physical activity Treatment group:", dfPA_treatment.shape)
print("Physical activity Balanced group: ", dfPA_balanced.shape)
print("Physical activity Control group: ", dfPA_control.shape)

Physical activity Treatment group: (7603, 10)
Physical activity Balanced group:  (1154, 10)
Physical activity Control group:  (1154, 10)


In [3]:
print("Balanced:")
print(dfPA_balanced.Age.describe())
print("Treatment:")
print(dfPA_treatment.Age.describe())

Balanced:
count    1154.000000
mean       56.643847
std         7.840486
min        45.000000
25%        51.000000
50%        55.500000
75%        61.000000
max        79.000000
Name: Age, dtype: float64
Treatment:
count    7603.000000
mean       56.692227
std         6.818423
min        23.000000
25%        53.000000
50%        57.000000
75%        61.000000
max        85.000000
Name: Age, dtype: float64


##### Create column outcome in categorical:

In [4]:
# Define the function to categorize PA_changes
def categorize_change(PA_change):
    if PA_change > 0:
        return 'Increased'
    elif PA_change < 0:
        return 'Decreased'
    else:
        return 'Unchanged'

# Apply the function to create a new column in df_PA_Balanced_before
dfPA_treatment['PA_change_category'] = dfPA_treatment['PA_change'].apply(categorize_change)
dfPA_balanced['PA_change_category'] = dfPA_balanced['PA_change'].apply(categorize_change)

##### Calculate Chi2:

In [5]:
dfPA_treatment['Group'] = 'Treatment'
dfPA_balanced['Group'] = 'Balanced'


df_combined = pd.concat([dfPA_treatment, dfPA_balanced])

# Create a contingency table for PA_change_category across Treatment and Balanced groups
contingency_table = pd.crosstab(df_combined['PA_change_category'], df_combined['Group'])

# Perform chi-squared test
chi2, p, dof, ex = chi2_contingency(contingency_table)

print(f"Chi-squared Test for PA_change_category across Treatment and Balanced groups")
print(f"Chi2: {chi2}, p-value: {p}, Degrees of Freedom: {dof}")
print("\nExpected Frequencies:")
print(ex)
print("\nObserved Frequencies:")
print(contingency_table)


Chi-squared Test for PA_change_category across Treatment and Balanced groups
Chi2: 8.454830331963484, p-value: 0.014590054719397184, Degrees of Freedom: 2

Expected Frequencies:
[[ 326.94689962 2154.05310038]
 [ 470.85097636 3102.14902364]
 [ 356.20212402 2346.79787598]]

Observed Frequencies:
Group               Balanced  Treatment
PA_change_category                     
Decreased                368       2113
Increased                442       3131
Unchanged                344       2359


#### Quality of life

In [6]:
dfQ_treatment = pd.read_csv("testdata/2.QOL_treatment.csv")
dfQ_control = pd.read_csv("testdata/2.QOL_control.csv")
dfQ_balanced = pd.read_csv("testdata/3.QOL_balanced.csv")

In [7]:
# List of file paths and corresponding dataframe names
files = [
    ("testdata/2.QOL_treatment.csv", "dfQ_treatment"),
    ("testdata/2.QOL_control.csv", "dfQ_control"),
    ("testdata/3.QOL_balanced.csv", "dfQ_balanced"),
]

# Dictionary to store dataframes
dataframes = {}

# Loop through each file and read the csv into a dataframe, then store it in the dictionary
for file, name in files:
    dataframes[name] = pd.read_csv(file)
    print(f"{name}: {dataframes[name].shape}")

dfQ_treatment: (7603, 10)
dfQ_control: (1154, 10)
dfQ_balanced: (1154, 11)


In [8]:
# Define the function to categorize PA_changes
def QOL_categorize_significant_change(QOL_change):
    if QOL_change >= 10:
        return 'Increased 10+ points'
    elif QOL_change <= -10:
        return 'Decreased 10+ points'
    else:
        return 'Maintained'

dfQ_treatment['QOL_change_category'] = dfQ_treatment['QOL_change'].apply(QOL_categorize_significant_change)
dfQ_control['QOL_change_category'] = dfQ_control['QOL_change'].apply(QOL_categorize_significant_change)
dfQ_balanced['QOL_change_category'] = dfQ_balanced['QOL_change'].apply(QOL_categorize_significant_change)

In [9]:
dfQ_balanced

,Age,Gender,BMI,Depression,Employment_status,pain_medication_baseline,Baseline_Physical_Activity,Baseline_Pain,Baseline_Quality_of_life,QOL_change,Treatment,QOL_change_category
0,74.0,1.0,29.6,1.0,1.0,0.0,1.0,70.0,50.00,-6.25,0,Maintained
1,49.0,2.0,33.8,1.0,1.0,0.0,1.0,60.0,56.25,-18.75,0,Decreased 10+ points
2,49.0,2.0,33.8,1.0,1.0,0.0,1.0,60.0,56.25,-18.75,0,Decreased 10+ points
3,49.0,2.0,33.8,1.0,1.0,0.0,1.0,60.0,56.25,-18.75,0,Decreased 10+ points
4,73.0,2.0,35.2,1.0,2.0,0.0,1.0,100.0,18.75,62.50,0,Increased 10+ points
...,...,...,...,...,...,...,...,...,...,...,...,...
1149,56.0,2.0,27.5,1.0,1.0,0.0,10.0,40.0,43.75,12.50,0,Increased 10+ points
1150,56.0,2.0,27.5,1.0,1.0,0.0,10.0,40.0,43.75,12.50,0,Increased 10+ points
1151,56.0,2.0,27.5,1.0,1.0,0.0,10.0,40.0,43.75,12.50,0,Increased 10+ points
1152,53.0,2.0,30.1,1.0,1.0,0.0,10.0,50.0,50.00,6.25,0,Maintained


In [10]:
# Add a new column to identify the group
dfQ_treatment['Group'] = 'Treatment'
dfQ_balanced['Group'] = 'Balanced'

# Combine the dataframes
dfQ_combined = pd.concat([dfQ_treatment, dfQ_balanced])

# Create a contingency table for QOL_change_category across Treatment and Balanced groups
contingency_table = pd.crosstab(dfQ_combined['QOL_change_category'], dfQ_combined['Group'])

# Perform chi-squared test
chi2, p, dof, ex = chi2_contingency(contingency_table)

# Display results
print(f"Chi-squared Test for QOL_change_category across Treatment and Balanced groups")
print(f"Chi2: {chi2}, p-value: {p}, Degrees of Freedom: {dof}")
print("\nExpected Frequencies:")
print(ex)
print("\nObserved Frequencies:")
print(contingency_table)


Chi-squared Test for QOL_change_category across Treatment and Balanced groups
Chi2: 31.306130631359743, p-value: 1.59206248381333e-07, Degrees of Freedom: 2

Expected Frequencies:
[[ 141.13669065  929.86330935]
 [ 546.22930227 3598.77069773]
 [ 466.63400708 3074.36599292]]

Observed Frequencies:
Group                 Balanced  Treatment
QOL_change_category                      
Decreased 10+ points       158        913
Increased 10+ points       458       3687
Maintained                 538       3003


#### Pain

In [11]:
dfpain_treatment = pd.read_csv("testdata/2.Pain_treatment.csv")
dfpain_control = pd.read_csv("testdata/2.Pain_control.csv")
dfpain_balanced = pd.read_csv("testdata/3.Pain_balanced.csv")

In [12]:
# List of file paths and corresponding dataframe names
files = [
    ("testdata/2.Pain_treatment.csv", "dfpain_treatment"),
    ("testdata/2.Pain_control.csv", "dfpain_control"),
    ("testdata/3.Pain_balanced.csv", "dfpain_balanced")
]

# Dictionary to store dataframes
dataframes = {}

# Loop through each file and read the csv into a dataframe, then store it in the dictionary
for file, name in files:
    dataframes[name] = pd.read_csv(file)
    print(f"{name}: {dataframes[name].shape}")


dfpain_treatment: (7603, 10)
dfpain_control: (1154, 10)
dfpain_balanced: (1154, 11)


In [13]:
# Define the function to categorize changes of at least 20 points
def pain_categorize_significant_change(pain_change):
    if pain_change <= -20:
        return 'Decreased 20+ points'
    elif pain_change >= 20:
        return 'Increased 20+ points'
    else:
        return 'Maintained'  # This now clearly means the pain changed but less than 20 points either way

dfpain_treatment['pain_change_category'] = dfpain_treatment['pain_change'].apply(pain_categorize_significant_change)
dfpain_control['pain_change_category'] = dfpain_control['pain_change'].apply(pain_categorize_significant_change)
dfpain_balanced['pain_change_category'] = dfpain_balanced['pain_change'].apply(pain_categorize_significant_change)


In [14]:
# Add a new column to identify the group
dfpain_treatment['Group'] = 'Treatment'
dfpain_balanced['Group'] = 'Balanced'

# Combine the dataframes
dfpain_combined = pd.concat([dfpain_treatment, dfpain_balanced])

# Create a contingency table for pain_change_category across Treatment and Balanced groups
contingency_table = pd.crosstab(dfpain_combined['pain_change_category'], dfpain_combined['Group'])

# Perform chi-squared test
chi2, p, dof, ex = chi2_contingency(contingency_table)

# Display results
print(f"Chi-squared Test for pain_change_category across Treatment and Balanced groups")
print(f"Chi2: {chi2}, p-value: {p}, Degrees of Freedom: {dof}")
print("\nExpected Frequencies:")
print(ex)
print("\nObserved Frequencies:")
print(contingency_table)


Chi-squared Test for pain_change_category across Treatment and Balanced groups
Chi2: 280.18615690906887, p-value: 1.4399555111880408e-61, Degrees of Freedom: 2

Expected Frequencies:
[[ 435.13851776 2866.86148224]
 [ 113.85817061  750.14182939]
 [ 605.00331164 3985.99668836]]

Observed Frequencies:
Group                 Balanced  Treatment
pain_change_category                     
Decreased 20+ points       310       2992
Increased 20+ points       267        597
Maintained                 577       4014
